# Azure聊天完成示例

本示例将介绍如何使用Azure OpenAI服务进行聊天完成，并包括有关内容过滤的信息。


## 设置

首先，我们安装必要的依赖项并导入我们将使用的库。


In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv


In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()


### 认证

Azure OpenAI 服务支持多种认证机制，包括 API 密钥和 Azure Active Directory 令牌凭据。


In [2]:
use_azure_active_directory = False  # 将此标志设置为 True，如果您正在使用 Azure Active Directory。


#### 使用API密钥进行身份验证

要设置OpenAI SDK以使用*Azure API密钥*，我们需要将`api_key`设置为与您的端点关联的密钥（您可以在[Azure门户](https://portal.azure.com)的*"资源管理"*下的*"密钥和端点"*中找到此密钥）。您还将在此处找到您资源的端点。


In [13]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2023-09-01-preview"
    )


#### 使用Azure Active Directory进行身份验证
现在让我们看看如何通过Azure Active Directory进行身份验证。我们将从安装`azure-identity`库开始。这个库将提供我们需要进行身份验证的令牌凭据，并帮助我们通过`get_bearer_token_provider`辅助函数构建一个令牌凭据提供程序。建议使用`get_bearer_token_provider`而不是向`AzureOpenAI`提供静态令牌，因为这个API会自动为您缓存和刷新令牌。

有关如何设置Azure Active Directory身份验证与Azure OpenAI的更多信息，请参阅[文档](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)。


In [ ]:
! pip install "azure-identity>=1.15.0"


In [5]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )


> 注意：如果未提供以下参数，则AzureOpenAI将从其对应的环境变量中推断出来：

- `api_key` 从 `AZURE_OPENAI_API_KEY`
- `azure_ad_token` 从 `AZURE_OPENAI_AD_TOKEN`
- `api_version` 从 `OPENAI_API_VERSION`
- `azure_endpoint` 从 `AZURE_OPENAI_ENDPOINT`


## 部署

在本节中，我们将创建一个GPT模型的部署，以便用于生成聊天完成。


### 部署：在Azure OpenAI Studio中创建
让我们部署一个模型以用于聊天完成。前往 https://portal.azure.com，找到您的Azure OpenAI资源，然后导航到Azure OpenAI Studio。点击“部署”选项卡，然后为您想要用于聊天完成的模型创建一个部署。您在模型中给出的部署名称将在下面的代码中使用。


In [4]:
deployment = "" # 在此处填写从门户获取的部署名称


## 创建聊天完成

现在让我们使用我们构建的客户端来创建一个聊天完成。


In [ ]:
# 关于所有可能的参数，请参阅 <https://platform.openai.com/docs/api-reference/chat-completions/create>。
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)

print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")


### 创建一个流式聊天完成

我们也可以流式传输响应。


In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta

        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)


### 内容过滤

Azure OpenAI 服务包括对提示和完成响应进行内容过滤。您可以在[这里](https://learn.microsoft.com/azure/ai-services/openai/concepts/content-filter)了解有关内容过滤以及如何配置的更多信息。

如果提示被内容过滤器标记，库将引发一个带有 `content_filter` 错误代码的 `BadRequestError` 异常。否则，您可以访问响应中的 `prompt_filter_results` 和 `content_filter_results`，查看内容过滤的结果以及哪些类别被标记。


#### 内容过滤器标记的提示


In [ ]:
import json

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "<text violating the content policy>"}
]

try:
    completion = client.chat.completions.create(
        messages=messages,
        model=deployment,
    )
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered!")
        content_filter_result = err["error"]["innererror"]["content_filter_result"]
        for category, details in content_filter_result.items():
            print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")


### 检查内容过滤器的结果


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the biggest city in Washington?"}
]

completion = client.chat.completions.create(
    messages=messages,
    model=deployment,
)
print(f"Answer: {completion.choices[0].message.content}")

# prompt content filter result in "model_extra" for azure
prompt_filter_result = completion.model_extra["prompt_filter_results"][0]["content_filter_results"]
print("\nPrompt content filter results:")
for category, details in prompt_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

# completion content filter result
print("\nCompletion content filter results:")
completion_filter_result = completion.choices[0].model_extra["content_filter_results"]
for category, details in completion_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")
